# Homework 10
#### Course Notes
**Language Models:** https://github.com/rjenki/BIOS512/tree/main/lecture17  
**Unix:** https://github.com/rjenki/BIOS512/tree/main/lecture18  
**Docker:** https://github.com/rjenki/BIOS512/tree/main/lecture19

## Question 1
#### Make a language model that uses ngrams and allows the user to specify start words, but uses a random start if one is not specified.

In [12]:
install.packages("httr")
install.packages("tokenizers")
install.packages("stringr")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘SnowballC’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



#### a) Make a function to tokenize the text.

In [1]:
tokenize_text <- function(text) {
    tokenizers::tokenize_words(text, lowercase=TRUE, strip_punct=TRUE)[[1]]
}


#### b) Make a function generate keys for ngrams.

In [2]:
key_from <- function(ngram, sep = "\x1f") {
    paste(ngram, collapse=sep)
}

#### c) Make a function to build an ngram table.

In [3]:
build_ngram_table <- function(tokens, n, sep = "\x1f") {
    if (length(tokens) < n) return(new.env(parent = emptyenv()))
    tbl <- new.env(parent = emptyenv())
    for (i in seq_len(length(tokens) - n + 1L)) {
        ngram <- tokens[i:(i + n - 2L)]
        next_word <- tokens[i + n - 1L]
        key <- paste(ngram, collapse = sep)
        counts <- if (!is.null(tbl[[key]])) tbl[[key]] else integer(0)
        if (next_word %in% names(counts)) {
            counts[[next_word]] <- counts[[next_word]] + 1L
        } else {
            counts[[next_word]] <- 1L
        }
        tbl[[key]] <- counts
    }
    tbl
}


#### d) Function to digest the text.

In [4]:
digest_text <- function(text, n) {
    tokens <- tokenize_text(text)
    build_ngram_table(tokens, n)
}

#### e) Function to digest the url.

In [5]:
digest_url <- function(url, n) {
    res <- httr::GET(url)
    txt <- httr::content(res, as = "text", encoding = "UTF-8")
    digest_text(txt,n)
}

#### f) Function that gives random start.

In [6]:
random_start <- function(tbl, sep = "\x1f") {
    keys <- ls(envir = tbl, all.names=TRUE)
    if (length(keys)==0) stop("No n-grams available. Digest text first.")
    picked <- sample(keys, 1)
    strsplit(picked, sep, fixed=TRUE)[[1]]
}

#### g) Function to predict the next word.

In [7]:
predict_next_word <- function(tbl, ngram, sep = "\x1f") {
    key <- paste(ngram, collapse = sep)
    counts <- if(!is.null(tbl[[key]])) tbl[[key]] else integer(0)
    if (length(counts) == 0) return(NA_character_)
    sample(names(counts), size=1, prob=as.numeric(counts))
}

#### h) Function that puts everything together. Specify that if the user does not give a start word, then the random start will be used.

In [8]:
make_ngram_generator <- function(tbl, n, sep = "\x1f") {
    force(tbl); n <- as.integer(n); force(sep)
    function(start_words = NULL, length = 10L) {
        if ((is.null(start_words)) || length(start_words) != n - 1L) {
            start_words <- random_start(tbl, sep=sep)
        }
        word_sequence <- start_words
        for (i in seq_len(max(0L, length - length(start_words)))) {
            ngram <- tail(word_sequence, n - 1L)
            next_word <- predict_next_word(tbl, ngram, sep=sep)
            if (is.na(next_word)) break
            word_sequence <- c(word_sequence, next_word)
        }
        paste(word_sequence, collapse= " ")
    }
}

## Question 2
#### For this question, set `seed=2025`.
#### a) Test your model using a text file of [Grimm's Fairy Tails](https://www.gutenberg.org/cache/epub/2591/pg2591.txt)
#### i) Using n=3, with the start word(s) "the king", with length=15.
#### ii) Using n=3, with no start word, with length=15.

In [20]:
set.seed(2025)

url <- "https://www.gutenberg.org/cache/epub/2591/pg2591.txt"
tbl3 <- digest_url(url, n = 3)
gen3 <- make_ngram_generator(tbl3, n = 3)
cat("i) With start words 'the king':\n")
print(gen3(length = 15, start_words = c("the", "king")))

i) With start words 'the king':
[1] "the king has forbidden me to marry another husband am not i shall ride upon"


In [19]:
set.seed(2025)

url <- "https://www.gutenberg.org/cache/epub/2591/pg2591.txt"
tbl3 <- digest_url(url, n = 3)
gen3 <- make_ngram_generator(tbl3, n = 3)
print(gen3(length = 15))

[1] "spread the jam over it spread its wings and crying here comes our hobblety jib"


#### b) Test your model using a text file of [Ancient Armour and Weapons in Europe](https://www.gutenberg.org/cache/epub/46342/pg46342.txt)
#### i) Using n=3, with the start word(s) "the king", with length=15.
#### ii) Using n=3, with no start word, with length=15.

In [21]:
set.seed(2025)

url <- "https://www.gutenberg.org/cache/epub/46342/pg46342.txt."
tbl3 <- digest_url(url, n = 3)
gen3 <- make_ngram_generator(tbl3, n = 3)
cat("i) With start words 'the king':\n")
print(gen3(length = 15, start_words = c("the", "king")))

i) With start words 'the king':
[1] "the king"


In [22]:
set.seed(2025)

url <- "https://www.gutenberg.org/cache/epub/46342/pg46342.txt."
tbl3 <- digest_url(url, n = 3)
gen3 <- make_ngram_generator(tbl3, n = 3)
print(gen3(length = 15))

[1] "ptrgc9io meta name google site verification content wucoevsnj5kp3ts_36ofp64laakk 1mvtg ptrgc9io meta name alexaverifyid content 4wnacljse"


#### c) Explain in 1-2 sentences the difference in content generated from each source.

For starters, it looks like the actual content in the Ancient Armour file contains meta data instead of actual narrative storytelling like the Fairy Tale file.

## Question 3


a) A language model is a probability distribution over words. You put some input into the probability distribution, which is conditioning the probability distribution, and you get an output from the probability distribution. It's basically predicts the next word (token) given the previous words.

b) You can use podman which is serverless and runs without root privileges.

## Question 4

| Term | Meaning |  
|------|---------|
| **Shell** | runs the mkdir project command  |
| **Terminal emulator** | the "place" the shell sits is the terminal emulator (HOSTS a shell). Displays inputs/outputs  |
| **Process** | something running on your computer which is the "mkdir" command  |
| **Signal** | things we can send to processes to tell them to do something  |
| **Standard input** | each process has these; they can read characters from the input and write to the output  |
| **Standard output** | the default output stream from a process  |
| **Command line argument** | the word "project" is the command line argument  |
| **The environment** |  all the stuff a process can see when its running; ok to imagine that when one process starts another, the child process sees all the same stuff as the parent  |

## Question 5


a) find, xargs, prep

b) This command first uses "find" to search the current directory for all R script files "*.R". The pipe "|" sends that list of files to "xargs", which then runs "grep read_csv" on each file. The final output shows every line in every R file where read_csv appears.

## Question 6


a) Unable to find image 'hello-worlddocker:latest' locally
docker: Error response from daemon: pull access denied for hello-worlddocker, repository does not exist or may require 'docker login'

b) docker run -it -p 8787:8787 rocker/verse

The password is set to ot5cheele6Oodein
If you want to set your own password, set the PASSWORD environment variable. e.g. run with:
docker run -e PASSWORD=<YOUR_PASS> -p 8787:8787 rocker/rstudio

c) Go to http://localhost:8787/ and put in the username "rstudio" and the password we just got.